In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import scipy.stats as stats
from sklearn.metrics import r2_score, accuracy_score, fbeta_score
from sklearn.model_selection import RandomizedSearchCV

df = pd.read_stata(r"D:\World Bank\Honduras PMT benchmark\Data_out\CONSOLIDADA_2023_clean.dta")

In [2]:
# df["EDAD2"] = df["EDAD"]**2
# oh1 = pd.get_dummies(df['CIVIL'], prefix="CIVIL_", dummy_na=True)
# oh2 = pd.get_dummies(df['CH307'], prefix="dis_", dummy_na=True)
# oh3 = pd.get_dummies(df['CH308'], prefix="orig_", dummy_na=True)
# oh4 = pd.get_dummies(df['OC609'], prefix="trab1_", dummy_na=True)
# oh5 = pd.get_dummies(df['CATEGOP'], prefix="trab2_", dummy_na=True)
# oh6 = pd.get_dummies(df['RAMAOP'], prefix="trab3_", dummy_na=True)
# oh7 = pd.get_dummies(df['OCUPAOP'], prefix="trab4_", dummy_na=True)
# oh8 = pd.get_dummies(df['DOMI'], prefix="domi_", dummy_na=True)
# oh9 = pd.get_dummies(df['SEXO'], prefix="genero_", dummy_na=True)
# oh10 = pd.get_dummies(df['ED01'], prefix="ed_", dummy_na=True)
# oh11 = pd.get_dummies(df['CA501'], prefix="ca501_", dummy_na=True)
# oh12 = pd.get_dummies(df['UR'], prefix="ur_", dummy_na=True)
# oh13 = pd.get_dummies(df['NBI'], prefix="nbi_", dummy_na=True)
# one_hots = pd.concat([oh1, oh2, oh3, oh4, oh5, oh6, oh7, oh8, oh9, oh10, oh11, oh12, oh13], axis=1)
# one_hots_cols = one_hots.columns.to_list()
# df = pd.concat([df, one_hots], axis=1)
# df = pd.concat([df], axis=1)

In [3]:
# Variables globales para las distintas estimaciones **
vars_pmtoriginal = ["Ocupacion_bien", "Paredes_bien", "Pension_bien", "Refri_mal", "Remesas_bien", "Aire_mal", "Alquileres_bien", "Alumbrado_bien", "Basura_bien", "Carro_mal", "Cocina2_bien", "Compu_mal", "Dependencia", "dv111", "Ed_diversif_bien", "Ed_univer_bien", "edad_0_5", "edad_15_21", "edad_60_120", "edad_6_14", "Estufa_mal", "Vivienda2_bien", "EqSonido_mal", "HaySanitario_bien", "Sanitario_bien", "Civil_mal", "Cocina_bien", "Cable_mal", "Hacinamiento", "Moto_mal", "Piso_mal", "Bici_mal", "Exterior_bien", "Dominio_1", "Dominio_2", "Dominio_3", "Vivienda_bien", "Agua2_bien", "Agua_bien", "TV_mal", "Telefono_mal"]
IPM_vars = ["privacion_agua_h", "privacion_saneamiento_h", "privacion_cocina_h", "privacion_educ_h", "privacion_asistencia_h", "privacion_alfab_h", "privacion_elec_h", "privacion_piso_h", "privacion_techo_h", "privacion_segsoc_h", "privacion_desocup_h", "privacion_subemp_h", "privacion_ocup_h", "privacion_trabinf_h", "privacion_trabadol1_h", "privacion_trabadol2_h", "privacion_pared_h", "privacion_hacina_h"]
vars_conjunto1 = vars_pmtoriginal + IPM_vars

#* Eliminamos las variables que eran muy multicolineales, eran falseables o no dependian de los hogares
vars_conjunto2 = ["privacion_saneamiento_h","privacion_cocina_h","privacion_educ_h","privacion_asistencia_h","privacion_alfab_h","Piso_mal","Paredes_bien","EqSonido_mal","HaySanitario_bien","Sanitario_bien","Cocina2_bien","Hacinamiento","Cable_mal","Moto_mal","Bici_mal","Dominio_1","Dominio_2","Dominio_3","Pension_bien","Refri_mal","Aire_mal","Carro_mal","Compu_mal","dv111","Ed_diversif_bien","Ed_univer_bien","edad_0_5","edad_15_21","edad_60_120","edad_6_14","Estufa_mal","Agua2_bien","Dependencia"]

In [4]:
df_test = df[df.test_set==1]
df_train = df[df.test_set==0]
X_test = df_test[vars_conjunto1]
y_test = df_test["logingreso"]
w_test = df_test["FACTOR_P"]
X_train = df_train[vars_conjunto1]
y_train = df_train["logingreso"]
w_train = df_train["FACTOR_P"]


In [5]:
df_train

,HOGAR,NPER,DOMINIO,DEPMUESTRA,COR_PRE,NUM_REC,NUM_HOG,V01,V02,V02OTRO,...,privacion_hacina_h,FACTOR_P,hogar,indice_pobreza_multi,pobreza_multidim,no_pob_multidim,log_ingreso_est,ingreso_est,random_num,test_set
0,133 1 10384 31381,1,Ciudades Medianas,Atlantida,10384,3,1,Casa individual,Madera al natural,,...,1.0,1061.000612,1,0.500000,1.0,0.0,6.275743,531.521079,0.973354,0.0
1,1257108080868131,1,Distrito Central,Francisco Morazan,80868,13,1,Casa individual,Otro,LAMINA DE ZINC,...,1.0,493.341042,1,0.541667,1.0,0.0,6.305543,547.598775,0.533083,0.0
3,732416160235081,1,Ciudades Pequeñas,Santa Barbara,160235,8,1,Casa individual,"Ladrillo, piedra o bloque",,...,0.0,1661.798779,1,0.083333,0.0,1.0,6.451044,633.363460,0.444628,0.0
4,934405053677081,1,Ciudades Pequeñas,Cortes,53677,8,1,Casa individual,"Ladrillo, piedra o bloque",,...,0.0,1546.433454,1,0.416667,1.0,0.0,6.474489,648.387745,0.885603,0.0
5,312406060730101,1,Ciudades Pequeñas,Choluteca,60730,10,1,Casa individual,"Ladrillo, piedra o bloque",,...,0.0,1876.150608,1,0.416667,1.0,0.0,6.518643,677.658082,0.897054,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16389,1451515080106106,1,Rural,Olancho,150801,6,1,Casa individual,Adobe,,...,0.0,310.292256,1,0.041667,0.0,1.0,NaN,NaN,0.335857,0.0
16390,851313014502125,1,Rural,Lempira,130145,2,1,Casa individual,Adobe,,...,0.0,379.998972,1,0.208333,0.0,1.0,NaN,NaN,0.908036,0.0
16391,417513130619051,1,Rural,Lempira,130619,5,1,Casa individual,Adobe,,...,0.0,276.732757,1,0.291667,1.0,0.0,NaN,NaN,0.970280,0.0
16393,832505052880091,1,Rural,Cortes,52880,9,1,Casa individual,"Ladrillo, piedra o bloque",,...,1.0,1314.773821,1,0.250000,0.0,1.0,NaN,NaN,0.623333,0.0


In [6]:
def fit_linear_model(df, y_var, vars):
    """ Fit a linear model to the data """
    
    from sklearn.linear_model import LinearRegression
    
    df_test = df[df.test_set==1]
    df_test = df_test[vars + [y_var, "FACTOR_P"]].dropna()
    df_train = df[df.test_set==0]
    df_train = df_train[vars + [y_var, "FACTOR_P"]].dropna()
    X_test = df_test[vars]
    y_test = df_test[y_var]
    w_test = df_test["FACTOR_P"]
    X_train = df_train[vars]
    y_train = df_train[y_var]
    w_train = df_train["FACTOR_P"]
    
    model = LinearRegression()
    model.fit(X_train, y_train, sample_weight=w_train)
    y_test_preds = model.predict(X_test)
    y_train_preds = model.predict(X_train)
        
    # Export dataframe with predictions
    df_test["logingreso_linear"] = y_test_preds
    df_train["logingreso_linear"] = y_train_preds
    df_out = pd.concat([df_test, df_train])
    df_out = df_out.sort_index()

    return df_out, model, y_test_preds, y_train_preds, y_test, y_train, w_test, w_train
    
    

def fit_xgboost_reg(df, y_var, vars, params_grid, scoring="neg_mean_absolute_error"):
    """
    Trains and evaluates an XGBoost regression model using a randomized grid search for hyperparameter tuning.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the dataset with both training and test sets, including features and target variables.
        The DataFrame should have a column 'test_set' where 0 indicates training data and 1 indicates test data.
    
    y_var: str
        A column name representing the variable to use as labels of the model.

    vars : list
        A list of column names representing the independent variables (features) used for training the model.
        
    params_grid : dict
        Dictionary where keys are XGBoost hyperparameters and values are lists of possible values for those
        hyperparameters. This grid is used for randomized hyperparameter tuning.

    Returns:
    --------
    random_search : RandomizedSearchCV object
        The fitted RandomizedSearchCV object containing the best estimator and results from the grid search.
        
    Workflow:
    ---------
    1. Splits the input DataFrame into training and test sets based on the 'test_set' column.
    2. Prepares the feature matrix (X) and target variable (y) for both training and test sets.
    3. Converts the data into DMatrix format required for XGBoost.
    4. Initializes an XGBoost regressor with the 'hist' tree method and 'cuda' for GPU acceleration.
    5. Conducts a RandomizedSearchCV to tune hyperparameters based on a given parameter grid.
    6. Fits the model to the training data and evaluates its performance on both the training and test sets.
    7. Prints R-squared metrics for both training and test sets.
    8. Returns the fitted RandomizedSearchCV object.
    """

    df_test = df[df.test_set==1]
    df_train = df[df.test_set==0]
    X_test = df_test[vars]
    y_test = df_test[y_var]
    w_test = df_test["FACTOR_P"]
    X_train = df_train[vars]
    y_train = df_train[y_var]
    w_train = df_train["FACTOR_P"]
    
    model = xgb.XGBRegressor(
        tree_method="hist",
        device="cuda",
    )

    random_search = RandomizedSearchCV(
        model, 
        param_distributions=params_grid, 
        n_iter=5, 
        cv=5, 
        scoring='neg_mean_absolute_error', 
        verbose=0, 
    )

    random_search.fit(X_train, y_train)
    y_test_preds = random_search.best_estimator_.predict(X_test)
    y_train_preds = random_search.best_estimator_.predict(X_train)
    
    # Export dataframe with predictions
    df_test["logingreso_xgboost"] = y_test_preds
    df_train["logingreso_xgboost"] = y_train_preds
    df_out = pd.concat([df_test, df_train])
    df_out = df_out.sort_index()
    
    return df_out, random_search, y_test_preds, y_train_preds, y_test, y_train, w_test, w_train

def asigna_beneficios(y, weights, percentage):
    df = pd.DataFrame({"y": y, "weights": weights})
    df = df.sort_values("y")

    ranking = df["weights"].cumsum()
    ranking_pct = ranking / ranking.max()    
    y_bool = (ranking_pct < percentage)    
    y_bool = y_bool.sort_index()
    
    return y_bool

def compute_metrics(y, y_preds, weights=None, percentage=.667):
    r_test = r2_score(y, y_preds, sample_weight=weights, multioutput='uniform_average')
    print(f"R2 test: {r_test}")

    y_bool = (y.values < np.quantile(y_preds, percentage))
    y_preds_bool = (y_preds < np.quantile(y_preds, percentage))

    acc_test = accuracy_score(y_bool, y_preds_bool, sample_weight=weights)
    print(f"Accuracy Test: {acc_test}")
    
    f2_test = fbeta_score(y_bool, y_preds_bool, beta=2, sample_weight=weights)
    print(f"F2 Test: {f2_test}")
    
    return r_test, acc_test, f2_test

def compute_metrics_urru(y_ur, y_preds_ur, y_ru, y_preds_ru, weights_ur=None, weights_ru=None, percentage_ru=.665468, percentage_ur=.6671753):

    # Create boolean arrays for each dataset
    y_bool_ru = asigna_beneficios(y_ru, weights_ru, percentage_ru)
    y_preds_bool_ru = asigna_beneficios(y_preds_ru, weights_ru, percentage_ru)
    y_bool_ur = asigna_beneficios(y_ur, weights_ur, percentage_ur)
    y_preds_bool_ur = asigna_beneficios(y_preds_ur, weights_ur, percentage_ur)

    # Create single arrays for all data
    y_bool = pd.concat([pd.Series(y_bool_ru), pd.Series(y_bool_ur)])
    y_preds_bool = pd.concat([pd.Series(y_preds_bool_ru), pd.Series(y_preds_bool_ur)])
    
    y = pd.concat([pd.Series(y_ru), pd.Series(y_ur)])
    y_preds = pd.concat([pd.Series(y_preds_ru), pd.Series(y_preds_ur)])
    weights = pd.concat([pd.Series(weights_ru), pd.Series(weights_ur)])

    r_test = r2_score(y, y_preds, sample_weight=weights, multioutput='uniform_average')
    print(f"R2 test: {r_test}")

    acc_test = accuracy_score(y_bool, y_preds_bool, sample_weight=weights)
    print(f"Accuracy Test: {acc_test}")
    
    f2_test = fbeta_score(y_bool, y_preds_bool, beta=2, sample_weight=weights)
    print(f"F2 Test: {f2_test}")
    
    return r_test, acc_test, f2_test


In [7]:
params_grid = {
    'learning_rate': stats.uniform(0.005, 0.1),
    'reg_alpha': stats.uniform(0, 0.1),
    'reg_lambda': stats.uniform(0.7, 0.3),
    'gamma':stats.uniform(0.1, 0.5),
    'max_depth': stats.randint(5, 30),
    'min_child_weight': stats.randint(5, 30),
    'n_estimators': stats.randint(50, 500),
    'colsample_bytree':stats.uniform(0.5, 0.5),
    'subsample': stats.uniform(0.5, .25),
}

casos = {
    "vars_all": {
        "df":df, 
        "vars": vars_conjunto1,
    },
    "vars_min": {
        "df":df, 
        "vars": vars_conjunto2,
    },

}

resultados = {}
for name, params in casos.items():
    print(f"Running model: {name}")
    df_model, vars = params.values()
    # # Nacional
    # results, y_test_preds, y_train_preds, y_test, y_train = fit_xgboost_reg(df_model, "logingreso", vars, params_grid)
    # r2_test, acc_test = compute_metrics(y_test, y_test_preds)
    # resultados[name + "_reg_nac"] = {}
    # resultados[name + "_reg_nac"]["df"] = df_model
    # resultados[name + "_reg_nac"]["results"] = results
    # resultados[name + "_reg_nac"]["best_params"] = results.best_params_
    # resultados[name + "_reg_nac"]["r_test"]  = r2_test
    # resultados[name + "_reg_nac"]["y_test_preds"] = y_test_preds

    # Modelo lineal (para garantizar que esté todo ok)
    df_out_ru, model_ru, y_test_preds_ru, y_train_preds_ru, y_test_ru, y_train_ru, w_test_ru, w_train_ru = fit_linear_model(df_model[df_model["UR"]=="Rural"], "logingreso", vars)
    df_out_ur, model_ur, y_test_preds_ur, y_train_preds_ur, y_test_ur, y_train_ur, w_test_ur, w_train_ur = fit_linear_model(df_model[df_model["UR"]=="Urbana"], "logingreso", vars)
    print("Linear Model")
    r2_test, acc_test, f2_test = compute_metrics_urru(y_test_ur, y_test_preds_ur, y_test_ru, y_test_preds_ru, weights_ur=w_test_ur, weights_ru=w_test_ru)
    df_out = pd.concat([df_out_ru, df_out_ur]).sort_index()
    df_out = df_out[["logingreso_linear"]].join(df_model[["HOGAR", "YPERHG", "TOTPER", "FACTOR", "UR", "indice_pobreza_multi", "pobreza", "pobreza_ext"]])
    resultados[name + "_linreg_urru"] = {}
    resultados[name + "_linreg_urru"]["df"] = df_out
    resultados[name + "_linreg_urru"]["model"] = (model_ru, model_ur)
    resultados[name + "_linreg_urru"]["best_params"] = {"Rural": (model_ru.intercept_, model_ru.coef_), "Urbana": (model_ur.intercept_, model_ur.coef_)}
    resultados[name + "_linreg_urru"]["r_test"]  = r2_test
    resultados[name + "_linreg_urru"]["f2_test"] = f2_test
    
    # Urbano y Rural estimados por separado
    df_out_ru, model_ru, y_test_preds_ru, y_train_preds_ru, y_test_ru, y_train_ru, w_test_ru, w_train_ru = fit_xgboost_reg(df_model[df_model["UR"]=="Rural"], "logingreso", vars, params_grid)
    df_out_ur, model_ur, y_test_preds_ur, y_train_preds_ur, y_test_ur, y_train_ur, w_test_ur, w_train_ur = fit_xgboost_reg(df_model[df_model["UR"]=="Urbana"], "logingreso", vars, params_grid)
    print("XGBoost Model")
    r2_test, acc_test, f2_test = compute_metrics_urru(y_test_ur, y_test_preds_ur, y_test_ru, y_test_preds_ru, weights_ur=w_test_ur, weights_ru=w_test_ru)
    df_out = pd.concat([df_out_ru, df_out_ur]).sort_index()
    df_out = df_out[["logingreso_xgboost"]].join(df_model[["HOGAR", "YPERHG", "TOTPER", "FACTOR", "UR", "indice_pobreza_multi", "pobreza", "pobreza_ext"]])

    resultados[name + "_xgboost_urru"] = {}
    resultados[name + "_xgboost_urru"]["df"] = df_out
    resultados[name + "_xgboost_urru"]["model"] = (model_ru, model_ur)
    resultados[name + "_xgboost_urru"]["best_params"] = {"Rural": model_ru.best_params_, "Urban": model_ur.best_params_}
    resultados[name + "_xgboost_urru"]["r_test"]  = r2_test
    resultados[name + "_xgboost_urru"]["f2_test"] = f2_test

    # results, y_test_preds, r_test, r_train, cm = fit_xgboost_cla(df_model, "pobreza", vars, params_grid)
    # resultados[name + "_cla"] = {}
    # resultados[name + "_cla"]["df"] = df_model
    # resultados[name + "_cla"]["results"] = results
    # resultados[name + "_cla"]["best_params"] = results.best_params_
    # resultados[name + "_cla"]["r_test"]  = r_test
    # resultados[name + "_cla"]["r_train"] = r_train
    # resultados[name + "_cla"]["confusion_matrix"] = cm
    # resultados[name + "_cla"]["y_test_preds"] = y_test_preds


Running model: vars_all
Linear Model
R2 test: 0.4190679871791275
Accuracy Test: 0.7821013880461709
F2 Test: 0.8365146038449838


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [10:29:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
C:\Users\Nico\AppData\Local\Temp\ipykernel_20260\1471250287.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `

XGBoost Model
R2 test: 0.4356632387235657
Accuracy Test: 0.7757733298743447
F2 Test: 0.8316333175174002
Running model: vars_min
Linear Model
R2 test: 0.39901290234450326
Accuracy Test: 0.7667862756314439
F2 Test: 0.8248207357898657


C:\Users\Nico\AppData\Local\Temp\ipykernel_20260\1471250287.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test["logingreso_xgboost"] = y_test_preds
C:\Users\Nico\AppData\Local\Temp\ipykernel_20260\1471250287.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["logingreso_xgboost"] = y_test_preds
C:\Users\Nico\AppData\Local\Temp\ipykernel_20260\1471250287.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

XGBoost Model
R2 test: 0.3977111773081161
Accuracy Test: 0.7618160790841089
F2 Test: 0.8212858949687621


In [12]:
resultados["vars_min_xgboost_urru"]["df"].to_stata(r"D:\World Bank\Honduras PMT benchmark\Data_out\Predicts_XGBoost.dta", index=False)